In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import Sequence , get_file
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler ,EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf


import matplotlib.pyplot as plt
import numpy as np
import cv2
import os, json, shutil
import os
import json
import sys
from datetime import datetime
import os
import time
from itertools import islice

import segmentation_models as sm
sm.set_framework('tf.keras')

Segmentation Models: using `tf.keras` framework.


In [2]:
out_test_data_dir = '/glade/scratch/hgamarro/Deep_learning/Final_Project//data/processed/Vegas_v1/test'

model_path = "/glade/scratch/hgamarro/Deep_learning/Final_Project/models/scratch/BD_v1_results_main/"

In [3]:
class DataGenerator(Sequence):
    def __init__(self, list_IDs,label_map , img_dir ,mode):
        'Initialization'
        self.list_IDs = list_IDs
        self.label_map = label_map
        self.on_epoch_end()
        self.img_dir = img_dir + "/images"
        self.mask_dir = img_dir + "/masks"
        self.mode = mode

    def __len__(self):
        return int(len(self.list_IDs))

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))    

    
    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index:(index+1)]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X, y
    
    
    def __data_generation(self, list_IDs_temp):
        """Generates data containing batch_size samples"""
        if self.mode == "train":
            # Generate data
            X, y = self.load_file(list_IDs_temp)
            return X, y
        elif self.mode == "val":
            X, y = self.load_file(list_IDs_temp)
            return X, y
        
        
    def load_file(self, id_list):
        list_IDs_temp = id_list
        for ID in list_IDs_temp:
            x_file_path = os.path.join(self.img_dir, ID)
            y_file_path = os.path.join(self.mask_dir, self.label_map.get(ID))
            # Store sample
            X = np.load(x_file_path)
            # Store class
            y = np.load(y_file_path).astype('float32')
        return X, y    
    

In [4]:
# ====================
# test set
# ====================

all_test_files = [s for s in os.listdir(out_test_data_dir + "/images/") if s.endswith('.npy')]
all_test_files.append([s for s in os.listdir(out_test_data_dir + "/masks/") if s.endswith('.npy')] )
test_image_label_map = {
        "image_file_{}.npy".format(i+1): "label_file_{}.npy".format(i+1)
        for i in range(int(len(all_test_files)))}
test_partition = [item for item in all_test_files if "image_file" in item]

test_generator= DataGenerator(test_partition,test_image_label_map,out_test_data_dir, "val")

In [5]:
BACKBONE = 'vgg16'
# BACKBONE = 'resnet50'
# BACKBONE = 'efficientnetb6'
model_name = 'Unet'
BATCH_SIZE = 32
LR = 0.0001
EPOCHS = 100

n_classes = 1  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model


if model_name == 'Unet':
    print(model_name)
    model = sm.Unet(BACKBONE,classes=n_classes,input_shape=(512, 512, 3), activation=activation)
elif model_name == 'Linknet':
    print(model_name)
    model = sm.Linknet(BACKBONE,classes=n_classes,input_shape=(512, 512, 3), activation=activation)
elif model_name == 'FPN':
    print(model_name)
    model = sm.FPN(BACKBONE,classes=n_classes,input_shape=(512, 512, 3), activation=activation)
    

# define optomizer
optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.Precision(threshold=0.5), sm.metrics.Recall(threshold=0.5), sm.metrics.FScore(threshold=0.5)  ]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)


Unet


In [6]:
model.load_weights(model_path + "vgg16_Unet.h5")
# model.load_weights(model_path + "vgg16_Linknet.h5")
# model.load_weights(model_path + "vgg16_FPN.h5")
# model.load_weights(model_path + "efficientnetb3_Unet.h5")

In [7]:
results = model.evaluate(test_generator)
print("test loss, test acc:", results)




6/6 [==============================] - 5s 819ms/step - loss: 0.1804 - iou_score: 0.8368 - precision: 0.9225 - recall: 0.9002 - f1-score: 0.9111
test loss, test acc: [0.18037061393260956, 0.8368191123008728, 0.9224523901939392, 0.9001668095588684, 0.9111483693122864]
